## AI Inference from a video file into a bunch of images
This notebook is an example how to use DeGirum PySDK to do AI inference of a video stream from a video file. The annotated frames are saved into new directory with `_annotated` suffix.


### Specify video file name here

In [ ]:
input_filename = "./images/Traffic.mp4" # video file to process

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools, cv2
from pathlib import Path

### Specify inference option here

In [ ]:
# Please uncomment and edit one of the following inference options to specify your system configuration case according to
# https://cs.degirum.com/doc/0.5.0/degirum.html#system-configuration-for-specific-use-cases

# 1. DeGirum Cloud Zoo inference:
#zoo = dg.connect_model_zoo("dgcps://cs.degirum.com", token=mytools.token_get())

# 2. AIServer inference via IP address using models from DeGirum Cloud model zoo
#zoo = dg.connect_model_zoo(("192.168.0.7", "https://cs.degirum.com/degirum_com/public"), token=mytools.token_get())

# 3. AIServer inference via IP address using local model zoo
#zoo = dg.connect_model_zoo("192.168.0.1")

# 4. ORCA board installed locally using models from DeGirum Cloud Model Zoo
#zoo = dg.connect_model_zoo("https://cs.degirum.com/degirum_com/public", token=mytools.token_get())

# 5. Local inference with locally deployed model
#zoo = dg.connect_model_zoo("full/path/to/model.json")

In [ ]:
# load object detection model
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v2_ssd_coco--300x300_quant_n2x_orca_1")

# set model parameters
model.image_backend = 'opencv' # select OpenCV backend: needed to have overlay image in OpenCV format
model.input_numpy_colorspace = 'BGR'
model.overlay_show_probabilities = True # show probabilities on overlay image
model.overlay_line_width = 1

In [ ]:
orig_path = Path(input_filename)
frame_path = orig_path.parent / (orig_path.name + "_original") # construct original images directory path

# decode video stream into a set of jpeg images
nframes = mytools.video2jpegs(input_filename, frame_path)
print(f"{nframes} images are saved into '{frame_path}'")

In [ ]:
ann_path = orig_path.parent / (orig_path.name + "_annotated") # construct annotated images directory path

if not ann_path.exists(): # create directory for annotated images
    ann_path.mkdir()
    
# AI prediction loop
fi = 0 # frame index
print("Processing frames:")
progress = mytools.Progress(nframes)
for res in model.predict_dir(str(frame_path)):
    fname = str(ann_path / f"frame_{fi:05d}.jpg")
    cv2.imwrite(fname, res.image_overlay)
    fi += 1
    progress.step()
print(f"{nframes} annotated images are saved into '{ann_path}'")